# 📊 중고 노트북 가격 크롤링 및 분석 프로젝트

---

### ✅ 문제 인식
- 중고 제품 가격이 **판매자 주관에 따라 결정**되다 보니, 합리적인 구매 판단이 어렵습니다.
- **신뢰할 수 있는 가격 데이터**가 실시간으로 정리되어 있지 않아 구매자/판매자 모두 기준 설정이 힘듭니다.

---

### ✅ 사용 기술 및 도구 선정 이유
| 도구 | 사용 목적 | 선정 이유 |
|------|-----------|------------|
| `Selenium` | 웹 자동화 및 데이터 크롤링 | 로그인 없이 접근 가능한 플랫폼에 적합 |
| `Fake UserAgent` | 봇 탐지 회피 | 반복 크롤링 시 차단 방지 |
| `pandas` | 데이터 정제 및 분석 | 표 형태로 다루기 편리 |
| `matplotlib`, `seaborn` | 데이터 시각화 | 가격 분포를 직관적으로 전달 |
| `Google Colab` | 실습 및 데모 환경 | 웹 기반으로 즉시 실행 가능하고 설치 부담 없음 |

---

### ✅ 기대 효과
- 지역별, 브랜드별 중고 노트북 시세를 빠르게 파악 가능
- 평균 가격 기준으로 **과하게 비싼 제품 혹은 저렴한 매물**을 필터링하여 정보 제공
- 사용자 맞춤형 자동화 분석 도구로 확장 가능 (예: 특정 브랜드 알림, 가격 추이 분석 등)

## 🛠️ 개발 환경 준비: 필수 라이브러리 설치

다음 `!pip` 명령어는 프로젝트 실행에 필요한 Python 패키지를 설치하기 위한 코드입니다.

In [ ]:
!pip install selenium chromedriver_autoinstaller fake_useragent gradio matplotlib seaborn

## ⚙️ 크롬 브라우저 & 드라이버 설치 (Google Colab 전용)

Colab 환경은 일반 PC와 달리 **크롬 브라우저나 드라이버가 설치되어 있지 않기 때문에**,  
웹 자동화를 위해 수동으로 설치해줘야 합니다.

다음은 크롬과 드라이버를 설치하고, Selenium이 인식할 수 있도록 경로를 등록하는 코드입니다.

### 코드 설명
| 코드 | 설명 |
|------------|--------------|
| `!apt-get update` | 시스템 패키지 목록을 최신 상태로 업데이트 |
| `!apt install chromium-browser chromium-chromedriver` | 크롬 브라우저와 크롬드라이버 설치 |
| `!ln -sf ...` | 드라이버를 Colab의 기본 실행 경로에 심볼릭 링크로 연결 (실행 경로 문제 해결) |
| `import os` | 운영체제(OS) 기능을 사용하기 위한 모듈 임포트 |
| `os.environ['PATH'] += ...` | 크롬과 드라이버 경로를 시스템 환경 변수에 등록하여 Selenium이 인식하도록 설정 |

In [ ]:
!apt-get update > /dev/null
!apt install chromium-browser chromium-chromedriver > /dev/null
!ln -sf /usr/bin/chromedriver /usr/local/bin/chromedriver

import os
os.environ['PATH'] += ':/usr/lib/chromium-browser/:/usr/bin/'

## 🔍 중고 노트북 가격 수집 및 분석 (코드 요약)

### 1. 환경 설정 및 크롬 드라이버 자동화
- `Selenium`, `FakeUserAgent`, `chromedriver_autoinstaller` 등을 이용해 웹 브라우저 자동화 환경 구성
- 크롤링 지역은 `'신당동'`, 스크롤은 3페이지까지 설정

---

### 2. 데이터 수집 (크롤링)
- 당근마켓에서 "노트북" 키워드로 검색된 게시글 정보를 수집
- **제목, 가격, 지역, 링크** 정보를 추출
- Selenium으로 페이지 스크롤하면서 동적으로 게시글 목록 크롤링

---

### 3. 데이터 정제 및 전처리
- `convert_price()` 함수를 통해 가격 텍스트에서 숫자만 추출
- `convert_date()`로 "몇일 전" 등의 텍스트를 실제 날짜로 변환
- `삼성`, `LG`, `그램`, `레노버`, `ASUS` 등 브랜드 키워드가 포함된 모델명 추출

---

### 4. 데이터프레임 변환 및 분석
- 크롤링한 데이터를 `pandas`의 `DataFrame`으로 변환
- 평균 가격 계산 후, **평균보다 20% 이상 비싼 제품 / 싼 제품**으로 나눠 필터링

---

### 5. 결과 출력
- 상위 가격 상품(Above Average)과 하위 가격 상품(Below Average) 각각 표로 출력
- 수집된 전체 데이터를 CSV로 저장 (`daangn_laptops.csv`)

---

### 6. 파일 다운로드 (Colab 환경)
- Google Colab 환경에서 자동으로 CSV 파일 다운로드 링크 제공

In [ ]:
import time
import re
import datetime
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from fake_useragent import UserAgent

import chromedriver_autoinstaller

region = '신당동'
max_page = 3

ua = UserAgent()
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument(f'user-agent={ua.random}')

chromedriver_autoinstaller.install()
driver = webdriver.Chrome(options=options)

region_param = region.replace(' ', '-') + '-28'
base_url = f'https://www.daangn.com/kr/buy-sell/?in={region_param}&search=노트북'
driver.get(base_url)

SCROLL_PAUSE_TIME = 2
data = []

for _ in range(max_page):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)

    try:
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'a[data-gtm="search_article"]'))
        )
    except:
        print("❌ 게시물 로딩 실패")
        continue

    items = driver.find_elements(By.CSS_SELECTOR, 'a[data-gtm="search_article"]')
    for item in items:
        try:
            title = item.find_element(By.CSS_SELECTOR, 'span.lm809sh').text
            price_text = item.find_element(By.CSS_SELECTOR, 'span.lm809si').text
            region_elem = item.find_element(By.CSS_SELECTOR, 'span.lm809sj').text
            link = 'https://www.daangn.com' + item.get_attribute('href')
            data.append([title, price_text, region_elem, link])
        except:
            continue

driver.quit()

def convert_price(price):
    price = re.sub(r'[^0-9]', '', price)
    return int(price) if price else None

def convert_date(text):
    now = datetime.datetime.now()
    if '일 전' in text:
        days = int(re.search(r'(\d+)일 전', text).group(1))
        return now - datetime.timedelta(days=days)
    elif '시간 전' in text:
        hours = int(re.search(r'(\d+)시간 전', text).group(1))
        return now - datetime.timedelta(hours=hours)
    else:
        return now

records = []
for title, price, date, link in data:
    model_match = re.findall(r'(삼성|LG|그램|레노버|ASUS|NT[0-9]+)', title)
    model = model_match[0] if model_match else '기타'
    records.append({
        'title': title,
        'model': model,
        'price': convert_price(price),
        'date': convert_date(date),
        'url': link
    })

df = pd.DataFrame(records)
print("총 게시글 수:", len(df))

if not df.empty:
    print("📌 예시:", df.iloc[0])
else:
    print("❗ 크롤링 결과가 비어 있음")

df.dropna(subset=['price'], inplace=True)

avg_price = df['price'].mean()
threshold_upper = avg_price * 1.2
threshold_lower = avg_price * 0.8

above_avg = df[df['price'] > threshold_upper]
below_avg = df[df['price'] < threshold_lower]

print("\n💸 평균 이상 가격 상품:")
display(above_avg[['title', 'price', 'url']])

print("\n🧨 평균 이하 가격 상품:")
display(below_avg[['title', 'price', 'url']])

df.to_csv("daangn_laptops.csv", index=False)
print("✅ CSV 파일 저장 완료: daangn_laptops.csv")

from google.colab import files
files.download("daangn_laptops.csv")

## 📊 시각화 구성 및 의미

Gradio를 통해 시각적으로 출력되는 구성 요소는 다음과 같습니다:

---

### 📈 **Price Histogram (가격 히스토그램)**  
**: 가격 분포를 보여주는 그래프**

- 각 중고 노트북 가격이 **어느 범위에 몰려 있는지 시각적으로 확인**할 수 있음
- 정규분포형인지, 치우쳐 있는지 등을 파악 가능  
- 예: 대부분의 노트북이 30~50만원대에 분포한다면, 평균 가격이 적정하다는 신뢰가 생김

---

### 📦 **Price Boxplot (가격 박스플롯)**  
**: 이상치와 가격 범위를 한눈에 볼 수 있는 그래프**

- **최소값, 최대값, 사분위수(Q1, Q2, Q3)**를 시각적으로 확인 가능
- 특히 **극단적으로 비싼 제품이나 싸게 올라온 매물**(outliers)을 빠르게 파악할 수 있음  
- 예: 200만원 이상 노트북이 박스 밖으로 튀어나온다면 해당 매물은 비정상적일 수 있음

---

### 💸 **Top 10 Above-Average Price Listings (평균 이상 상위 10개 상품)**  
**: 평균 가격보다 비싼 매물 TOP 10 리스트**

- 가격이 평균보다 **20% 이상 높은 상품만 필터링**
- 프리미엄 제품, 신제품급 중고, 허위매물 의심 항목 등을 선별할 때 유용

---

### 🧨 **Top 10 Below-Average Price Listings (평균 이하 하위 10개 상품)**  
**: 평균 가격보다 저렴한 매물 TOP 10 리스트**

- 가격이 평균보다 **20% 이상 낮은 매물**
- **가성비 좋은 제품**, 빠른 거래를 원하는 매물 등 **실구매자에게 유용한 정보** 제공
- 예: 평균가 70만 원일 때, 50만 원 이하 제품만 표시됨

---

## ✨ 요약

| 시각화 항목 | 설명 | 목적 |
|-------------|------|------|
| 가격 히스토그램 | 가격 분포 시각화 | 평균가, 중심 범위 파악 |
| 가격 박스플롯 | 이상치 및 범위 표현 | 극단값 및 신뢰도 판단 |
| 평균 이상 TOP 10 | 비싼 매물 선별 | 프리미엄/비정상 매물 확인 |
| 평균 이하 TOP 10 | 저렴한 매물 선별 | 가성비 추천 또는 급매 탐색 |

In [ ]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import io

def visualize_laptop_data(file):
    df = pd.read_csv(file)
    if 'price' not in df.columns:
        return "Missing 'price' column in CSV."

    df.dropna(subset=['price'], inplace=True)

    fig1, ax1 = plt.subplots(figsize=(8, 4))
    sns.histplot(df['price'], bins=30, kde=True, ax=ax1)
    ax1.set_title("Laptop Price Histogram")
    ax1.set_xlabel("Price (KRW)")
    ax1.set_ylabel("Frequency")
    buf1 = io.BytesIO()
    fig1.savefig(buf1, format="png")
    plt.close(fig1)

    fig2, ax2 = plt.subplots(figsize=(8, 2))
    sns.boxplot(x=df['price'], ax=ax2)
    ax2.set_title("Laptop Price Boxplot")
    buf2 = io.BytesIO()
    fig2.savefig(buf2, format="png")
    plt.close(fig2)

    with open("hist.png", "wb") as f:
        f.write(buf1.getvalue())
    with open("box.png", "wb") as f:
        f.write(buf2.getvalue())

    avg_price = df['price'].mean()
    threshold_upper = avg_price * 1.2
    threshold_lower = avg_price * 0.8

    above_avg = df[df['price'] > threshold_upper][['title', 'price', 'url']].reset_index(drop=True)
    below_avg = df[df['price'] < threshold_lower][['title', 'price', 'url']].reset_index(drop=True)

    return (
        "hist.png",
        "box.png",
        above_avg.head(10),
        below_avg.head(10),
    )

iface = gr.Interface(
    fn=visualize_laptop_data,
    inputs=gr.File(label="Upload CSV File (e.g. daangn_laptops.csv)"),
    outputs=[
        gr.Image(type="filepath", label="Price Histogram"),
        gr.Image(type="filepath", label="Price Boxplot"),
        gr.Dataframe(label="Top 10 Above-Average Price Listings"),
        gr.Dataframe(label="Top 10 Below-Average Price Listings")
    ],
    title="Used Laptop Price Analysis",
    description="Upload a CSV file to visualize price distribution and compare with the average."
)

iface.launch()